<a href="https://colab.research.google.com/github/zyferion/privacyguardians/blob/main/notebooks/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction Notebook

This notebook explores function that can scrape and extract separate sections of text from websites relating to 'Privacy Policy' and or 'Terms and Conditions' (this includes 'Terms of Service', 'Terms of Use', etc.)

A key challenge with this work is that different websites have different formats:
- sometimes these sites have numbered section headings (either X. or X.Y)
- sometimes they do not have any numbered section headings and are separated by a html header tag (i.e., h2, h3)
- sometimes the website format is tricky -> div panels, javascript


In [2]:
# Read in required packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [3]:
# Read in raw file from GitHub
url = 'https://raw.githubusercontent.com/zyferion/privacyguardians/main/data/urls_dataset.csv'

df = pd.read_csv(url)# Dataset is now stored in a Pandas Dataframe

In [4]:
df.head(10)

,application_name,document_type,url,text_format
0,netflix,privacy policy,https://help.netflix.com/en/legal/privacy,h_sections
1,netflix,terms and conditions,https://help.netflix.com/en/legal/termsofuse,num_sections_2
2,airbnb,privacy policy,https://www.airbnb.com.au/help/article/3175,num_sections_2
3,airbnb,terms and conditions,https://www.airbnb.com.au/help/article/2908,num_sections_2
4,whatsapp,privacy policy,https://www.whatsapp.com/legal/privacy-policy,h_sections
5,whatsapp,terms and conditions,https://www.whatsapp.com/legal/terms-of-service,h_sections
6,temu,privacy policy,https://www.temu.com/au/privacy-and-cookie-pol...,h_sections
7,temu,terms and conditions,https://www.temu.com/au/terms-of-use.html,num_sections_2
8,7plus,terms and conditions,https://support.7plus.com.au/hc/en-au/articles...,num_sections_1
9,7plus,privacy policy,https://www.sevenwestmedia.com.au/privacy-poli...,num_sections_1


In [5]:
# Count of URLs
len(list(df['url']))

35

**A function for websites that DO NOT have numbered sections and are identified by h3 or h2 tag**
* Does not work on everything

In [6]:
# Given URL return pandas df with a row per section
# Only works on websites where a h3 or h2 tag separates each section

def scrape_and_store_h3_h2_text(url):
    data = []  # List to store extracted data

    # Send a GET request to the URL
    response = requests.get(url)
    if response.status_code == 200:  # If the request is successful
        soup = BeautifulSoup(response.text, "html.parser")  # Create a BeautifulSoup object 'soup'
        h3_tags = soup.find_all(["h3", "h2"])  # Find all <h3> and <h2> tags
        for h_tag in h3_tags:
            section_text = ""
            next_sibling = h_tag.find_next_sibling()  # Get the next sibling of the <h3> or <h2> tag
            while next_sibling and (next_sibling.name not in ["h3", "h2"]):
                section_text += str(next_sibling)  # Include the entire tag content
                next_sibling = next_sibling.find_next_sibling()  # Move to the next sibling
            if section_text:
                section_text_cleaned = BeautifulSoup(section_text, "html.parser").get_text()  # Clean HTML content
                data.append({"URL": url, "Section": h_tag.get_text(), "Text": section_text_cleaned})
    else:
        print("Failed to retrieve the page:", url)

    df = pd.DataFrame(data)  # Create a DataFrame from the collected data
    return df

In [14]:
# https://help.netflix.com/en/legal/privacy
# https://www.whatsapp.com/legal/privacy-policy
# https://www.airbnb.com.au/help/article/3175 (numbered)

#URL = "https://bumble.com/en-us/terms"

# Example
#if __name__ == "__main__":
#    url = URL  # Replace with the actual URL
#    df = scrape_and_store_h3_h2_text(url)
#    if not df.empty:
#        pd.set_option("display.max_colwidth", None)  # Display full content of DataFrame columns

# Show result
#df

In [7]:
# Store URLs in a list
url_list = list(df['url'])

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop through the URLs list and scrape data
for url in url_list:
    scraped_data = scrape_and_store_h3_h2_text(url)
    result_df = result_df.append(scraped_data, ignore_index=True)


<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append metho

Failed to retrieve the page: https://support.7plus.com.au/hc/en-au/articles/360060194292-Terms-and-Conditions


<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(scraped_data, ignore_index=True)
<ipython-input-7-db9f8bba9f14>:10: FutureWarning: The frame.append metho

In [8]:
# View results
result_df.head(10)

,URL,Section,Text
0,https://help.netflix.com/en/legal/privacy,Contacting Us\r,If you have general questions about your accou...
1,https://help.netflix.com/en/legal/privacy,Collection of Information\r,We receive and store information about you suc...
2,https://help.netflix.com/en/legal/privacy,Use of Information\r,"We use information to provide, analyze, admini..."
3,https://help.netflix.com/en/legal/privacy,Disclosure of Information\r,We disclose your information for certain purpo...
4,https://help.netflix.com/en/legal/privacy,Access to Account and Profiles\r,“Remember me” function: For ease of access t...
5,https://help.netflix.com/en/legal/privacy,Your Choices\r,Email and Text Messages. If you no longer want...
6,https://help.netflix.com/en/legal/privacy,Cookies and Internet Advertising\r,We and our Service Providers use cookies and o...
7,https://help.netflix.com/en/legal/privacy,Your Information and Rights,You can request access to your personal inform...
8,https://help.netflix.com/en/legal/privacy,Security\r,"We use reasonable administrative, logical, phy..."
9,https://help.netflix.com/en/legal/privacy,"Other Websites, Platforms and Applications\r ...",The Netflix service may be provided through an...


In [15]:
# Summary info on results
result_df.describe()

,URL,Section,Text
count,559,559,559
unique,28,496,535
top,https://policies.google.com/privacy,,
freq,89,30,4


In [17]:
# Identify missing URls from original set
difference_set = set(df['url'].drop_duplicates()) - set(result_df['URL'].drop_duplicates())

difference_list = list(difference_set)
difference_list

['https://explore.zoom.us/en/terms/',
 'https://web.didiglobal.com/au/legal/passenger-agreement/',
 'https://sheingroup.com/terms-conditions/',
 'https://support.7plus.com.au/hc/en-au/articles/360060194292-Terms-and-Conditions',
 'https://web.didiglobal.com/au/legal/privacy-policy/',
 'https://sheingroup.com/privacy-policy/']

**A function to handle sections separated by numbers**

In [130]:
def scrape_section_to_dataframe(url):
    # Send an HTTP request to the URL and get the HTML content
    response = requests.get(url)
    html_content = response.text

    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Define the section pattern as '1.1'
    section_pattern = r'\d+\.\d+'

    # Initialize lists to store data
    urls = []
    section_patterns = []
    section_texts = []

    matching_elements = soup.find_all(text=re.compile(section_pattern))

    for element in matching_elements:
        section_text = element.find_next().text.strip()

        urls.append(url)
        section_patterns.append(section_pattern)
        section_texts.append(section_text)

    data = {'URL': urls, 'Section': section_patterns, 'Text': section_texts}
    df = pd.DataFrame(data)
    return df


In [131]:
scrape_section_to_dataframe('https://web.didiglobal.com/au/legal/passenger-agreement/')

<ipython-input-130-072664d1e21c>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  matching_elements = soup.find_all(text=re.compile(section_pattern))


,URL,Section,Text
0,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,.gatsby-image-wrapper{position:relative;overfl...
1,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,(a) be over 18 years old and legally able to e...
2,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,1.3 You acknowledge and agree that DiDi will c...
3,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,1.4 You warrant that the information provided ...
4,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,1.5 DiDi reserves the right to refuse registra...
...,...,...,...
68,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,2.3 If another passenger suffers loss or prope...
69,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,3. Respect
70,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,(a) be on time for your ride – always try to b...
71,https://web.didiglobal.com/au/legal/passenger-...,\d+\.\d+,3.3 We reserve the right to deactivate your ac...


## Text analysis

In [50]:
# Function to count words in a text string
def count_words(text):
    words = text.split()
    return len(words)

# Apply the function to the 'text' column and calculate average
word_count= result_df['Text'].apply(count_words)
avg_word_count = word_count.mean()

# Show result
avg_word_count

247.13774597495527